In [1]:
from cerebellum.data_prep.seg_prep import *
from cerebellum.block_chain.block_lock import *

import json

resolution = (30, 48, 48)
block_size = 60
affinity_offset = 14
wz_thresh = 0.5
n_blocks = 15
sblock_ids = range(0,n_blocks-1)

/home/srujanm/anaconda2/envs/em/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# link blocks in linear fashion
start_time = time.time()
for sblock_id in sblock_ids:
    tblock_id = sblock_id + 1
    # load sblock
    zz_sb = sblock_id*block_size + affinity_offset
    sblock_name = "waterz%.2f-48nm-crop2gt-%04d"%(wz_thresh, zz_sb)
    sblock = SegPrep(sblock_name, resolution)
    if sblock_id==0: sblock.read_internal(stage="filtered")
    else: sblock.read_internal(stage="locked-to-0")
    sblock.read_bboxes() # Warning! If objects are relabeled, load relabeled-bboxes.json
    sblock_seg = sblock.data
    sbbox_dict = sblock.bbox_dict
    # load tblock
    zz_tb = tblock_id*block_size + affinity_offset
    tblock_name = "waterz%.2f-48nm-crop2gt-%04d"%(wz_thresh, zz_tb)
    tblock = SegPrep(tblock_name, resolution)
    tblock.read_internal(stage="filtered")
    tblock.read_bboxes() # Warning! If objects are relabeled, load relabeled-bboxes.json
    tblock_seg = tblock.data
    tbbox_dict = tblock.bbox_dict
    print "Linking block %d and %d"%(sblock_id, tblock_id)
    tblock_locked = block_lock(sblock_seg, tblock_seg, iou_thresh=0.5, 
                               tbbox_dict=tbbox_dict)
    tblock.data = tblock_locked
    tblock.write(stage="locked-to-0")
print "Completed block-chain. Total runtime: %f"%(time.time()-start_time)

Linking block 0 and 1
Runtime: 76.827350
Linking block 1 and 2
Runtime: 33.749339
Linking block 2 and 3
Runtime: 81.018039
Linking block 3 and 4
Runtime: 95.126621
Linking block 4 and 5
Runtime: 77.094146
Linking block 5 and 6
Runtime: 77.558642
Linking block 6 and 7
Runtime: 76.232953
Linking block 7 and 8
Runtime: 78.256160
Linking block 8 and 9
Runtime: 77.706153
Linking block 9 and 10
Runtime: 86.197607
Linking block 10 and 11
Runtime: 72.770326
Linking block 11 and 12
Runtime: 72.838658
Linking block 12 and 13
Runtime: 71.199968
Linking block 13 and 14
Runtime: 32.941181
Completed block-chain. Total runtime: 1053.310320
